# NLP重点知识点回顾

* 1 one-hot编码 

In [4]:
#// An highlighted block
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)

['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']


In [5]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)


[0 0 2 0 1 1 2 0 2 1]


In [7]:
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)


[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [8]:
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

['cold']


* 为什么要进行one-hot编码
one-hot编码类似将中文的item（gram或者简单理解为词语）进行统一编码，在某种程度上可以看做是词语的有效索引。但抛开one-hot编码存在稀疏矩阵，当词典长度存在较大，除了引起维度灾难之外，更应该关注的是，其无法表示词语与词语之间的相似程度。比如在一条句子中，梦想和理想可能在句子中的距离挨的很远，在另一份句子中离得却很近，但这种变化关系，ont-hot是无法体现出的。

* 2 word2vec

> word2vec又称为词向量，英文名称为word embedding，也可称为词嵌入。不使用one-hot编码的原因是one-hot无法准确表达不同词之间的相似度，像余弦相似度，Tf-idf可能会用这个。

> word2vec将每个词表示为一个定长的向量，并使得这些向量能较好地表现不同词之间的相似和类比关系。

> * Skip-gram模型：

$$正常句子：the~man ~loves ~his ~son $$
$$ 中心词为loves：~P('the','man','his','son'|'loves')=P('the'|'loves')*P('man'|'loves')*P('his'|'loves')*P('son'|'loves')$$

> 上述成立条件在词与词之间相互独立的条件下成立

> 使用softmax函数对上式进行表征=>给定中心词生成背景词的条件概率

$$ P(w_o|w_c)= \frac{\exp(u_o^T*v_c)}{\sum_{i \in \Upsilon}\exp{u_i^T}{v_c}} $$

>> $w_o$为背景词，索引为o，$w_c$为中心词，索引为c。该词在词典中的索引为i，当它为中心词时词向量为$v_i$，当它为背景词时词向量为$u_i$

> Q1：为什么要使用softmax函数？
* 1 信息论-从信息论的角度看，Softmax函数可以看作是试图最小化预测与事实之间的交叉熵。
> * 2 概率观点-从这个角度来看，我们实际上是在观察对数概率，因此，当我们进行幂运算时，我们最终得到的是原始概率。在这种情况下，Softmax方程找到了最大似然估计(MLE)。

> Q2：softmax函数具有哪些特点？
* 1 softmax可以最大化向量中的某个较大的值，同时又不失去较小值的可能性，符合概率的思想，并且对于一个向量，softmax结果之和等于1;
* 2 softmax交叉熵目标函数与最大化似然估计同宗同脉

> * 当给定一个长度为T的文本序列时，设时间步t的词为w(t)。假设给定中心词的情况下背景词的生成相互独立，当背景窗口大小为m时，skip-gram模型的似然函数——即给定任一中心词生成所有背景词的概率是：

$$\prod_{t=1}^{T}\prod_{~-m \le j \le m, j \neq0}P(w^{(t+j)}|w^{t})$$

> 神经网络就是为了获得合适的权重，这个权重也可称为词向量权重。而我们训练skip-gram网络，其重要的一点就是将对上式的最大似然估计，等价为对如下损失函数的最小化：

$$- \prod_{t=1}^{T}\prod_{~-m \le j \le m, j \neq0}\log P(w^{(t+j)}|w^{t})$$

> Q3：为什么是最大似然估计?
* 1 似然估计是指某个事件发生的概率受到可变参数的影响，当这些参数发生变化时，获得某个事件结果的概率也随之变化。
* 2 最大似然估计就是获得某个事件出现某个结果的最大可能性的那个最优参数。具体可以参考下面的表达式：
$$ \prod_{i=1}^{n}f(x_i;\theta_1,\theta_2,...\theta_n) $$

> 上式表示某个事件X的N次测量结果相互独立，其概率密度函数f(x_i)在已知样本X1，X2，Xn的情况产生的联合概率密度函数受到```\theta```的影响，并且称之为似然函数。似然的意思就是可能性

> 最大似然估计是指给定样本值X1，X2，Xn，如果有$$\prod f(x_i;\hat{\theta_1},\hat{\theta_2},...\hat{\theta_n},) \ge \prod f(x_i;\theta_1',\theta_2',...\theta_n')$$

> 其中$\hat{\theta_1}...$为未知参数$\theta_1$可能取的某一组值，而$\theta_1'$为其他可以取的一切值。大于号说明左边取值获得的可能性更大，如果存在这样的一组值使上式成立，我们就称左边\theta取值是最大化样本概率的值

> 求样本的最大似然估计的最优参数 $\theta$一般采用微分方法（除了微分也有其他方法），通过极大似然函数对$\theta$求偏导，令其等于零可以求出其解析最优解。但机器学习的方法则是通过已知样本进行近似一组数值解，使目标函数尽可能小。

> 我们对损失函数$- \prod_{t=1}^{T}\prod_{~-m \le j \le m, j \neq0}\log P(w^{(t+j)}|w^{t})$按照概率函数$ P(w_0|w_c)= \frac{\exp(u_o^T*v_c)}{\sum_{i \in \Upsilon}\exp{u_i^T}{v_c}} $带入，可以获得下面的式子：

$$ \prod \prod (\log{\exp(u_o^T*v_c)}-\log(\sum \exp(u_i^T*v_c)))$$

> 化简可得，$\prod\prod (u_o^Tv_c-\log(\sum(\exp(u_i^Tv_c)))$

> 而对$v_c$进行求偏导，可以获得

$$ \frac {\partial \log P(w_o|w_c)} {\partial v_c} = u_o-\frac {\sum \exp(u_j^Tv_c)}{\sum{\exp(u_i^T*v_c)}}*u_j\\
=u_o - \sum (\frac {\exp{u_j^Tv_c}} {\sum \exp{u_i^Tv_c}})*u_j \\
= u_o - \sum_{j \in \Upsilon} P(w_j|w_c)*u_j$$

>> 前面的两个连乘计算是训练文本的时候，进行权重更新使用的，第一个连乘代表长度为T的句子，该句子中的每个单词都要作为中心词去跟词典中的$u_j$去做softmax运算；第二个连乘代表中心词窗口为m的m个式子相乘，$u_o$表示窗口中的背景词的词向量，$v_c$表示窗口中的中心词的词向量，$u_i$和$u_j$是指词典$\Upsilon$中每个单词的背景词向量。

> Q4：$u_o和u_j$的表示形式是什么？  
词向量

> Q5：为什么求导会又出现一个$u_j$？
$$ \begin{eqnarray*}\\
&&\frac{\partial \ln \sum_{i \in \Upsilon} \exp(u_i*v_c)}{\partial v_c} \\
&=&\frac{\frac{\partial[\sum_{i \in \Upsilon} \exp(u_i^T*v_c)]}{\partial v_c}}{\sum_{i \in \Upsilon} \exp(u_i^T*v_c)}\\
&=& \frac{\frac{\partial [\exp(u_1^T*v_c) + \exp(u_2^T*v_c)+...\exp(u_\Upsilon^T*v_c)]}{\partial v_c}}{\sum_{i \in \Upsilon} \exp(u_i^T*v_c)}\\
&=& \frac{[\exp(u_1^T*v_c)*u_1 + \exp(u_2^T*v_c)*u_2+...\exp(u_\Upsilon^T*v_c)*u_\Upsilon]}{\sum_{i \in \Upsilon} \exp(u_i^T*v_c)}\end{eqnarray*}$$

> 为了避免产生混乱，分子不能再使用i进行累加表示，所以需要换索引j，最终表示为：
$$ \begin{eqnarray*}\\
&& \frac{\sum_{j \in \Upsilon}\exp(u_j^T*v_c)*u_j}{\sum_{i \in \Upsilon}\exp(u_i^T*v_c)}\\
&=& \sum_{j \in \Upsilon} \frac {\exp(u_j^T*v_c)*u_j} {\sum_{i \in \Upsilon}\exp(u_i^T*v_c)}\end{eqnarray*}$$

In [6]:
import numpy as np
import random

In [7]:
def softmax(x):
    origin_shape = x.shape
    # 根据输入的是矩阵还是向量分别计算softmax
    if len(x.shape) > 1:
        # 矩阵
        tmp = np.max(x,axis=1) # 得到每行的最大值，用于缩放每行的元素，避免溢出
        x-=tmp.reshape((x.shape[0],1)) # 使每行减去所在行的最大值（广播运算）

        x = np.exp(x) # 第一步，计算所有值以e为底的x次幂
        tmp = np.sum(x, axis = 1) # 将每行求和并保存
        x /= tmp.reshape((x.shape[0], 1)) # 所有元素除以所在行的元素和（广播运算）
    else:
        # 向量
        tmp = np.max(x) #最大值
        x -= tmp # 利用最大值缩放数据
        x = np.exp(x) # 对所有元素求以e为底的x次幂
        tmp = np.sum(x)  # 求元素和
        x /= tmp  # 求softmax
    return x

In [8]:
def sigmoid(x):
    s = np.true_divide(1, 1 + np.exp(-x)) # 使用np.true_divide进行加法运算
    return s

In [9]:
def sigmoid_grad(s):
    ds = s * (1 - s) # 可以证明：sigmoid函数关于输入x的导数等于`sigmoid(x)(1-sigmoid(x))`
    return ds

In [10]:
def softmaxCostAndGradient(predicted, target, outputVectors):
    v_hat = predicted # 中心词向量
    z = np.dot(outputVectors, v_hat) # 预测得分
    y_hat = softmax(z) # 预测输出y_hat
    
    cost = -np.log(y_hat[target]) # 计算代价

    z = y_hat.copy()
    z[target] -= 1.0
    grad = np.outer(z, v_hat) # 计算中心词的梯度
    gradPred = np.dot(outputVectors.T, z) # 计算输出词向量矩阵的梯度

    return cost, gradPred, grad

In [11]:
def skipgram(currentWord, contextWords, tokens, inputVectors, outputVectors):
    # 初始化变量
    cost = 0.0
    gradIn = np.zeros(inputVectors.shape)
    gradOut = np.zeros(outputVectors.shape)
    
    cword_idx = tokens[currentWord] # 得到中心单词的索引
    v_hat = inputVectors[cword_idx] # 得到中心单词的词向量

    # 循环预测上下文中每个字母
    for j in contextWords:
        u_idx = tokens[j] # 得到目标字母的索引
        c_cost, c_grad_in, c_grad_out = softmaxCostAndGradient(v_hat, u_idx, outputVectors) #计算一个中心字母预测一个上下文字母的情况
        cost += c_cost # 所有代价求和
        gradIn[cword_idx] += c_grad_in # 中心词向量梯度求和
        gradOut += c_grad_out # 输出词向量矩阵梯度求和

    return cost, gradIn, gradOut

In [12]:
inputVectors = np.random.randn(5, 3) # 输入矩阵，语料库中字母的数量是5，我们使用3维向量表示一个字母

In [13]:
outputVectors = np.random.randn(5, 3) # 输出矩阵

In [14]:
sentence = ['a', 'e', 'd', 'b', 'd', 'c','d', 'e', 'e', 'c', 'a'] # 句子

In [15]:
centerword = 'c' # 中心字母

In [16]:
context = ['a', 'e', 'd', 'd', 'd', 'd', 'e', 'e', 'c', 'a'] # 上下文字母

In [17]:
tokens = dict([("a", 0), ("b", 1), ("c", 2), ("d", 3), ("e", 4)]) # 用于映射字母在输入输出矩阵中的索引

In [18]:
c, gin, gout = skipgram(centerword, context, tokens, inputVectors, outputVectors)

In [28]:
print('cost:\n',c)

cost:
 19.72078784987426


In [19]:
step = 0.01  # 更新步进 

In [20]:
print('原始输入矩阵：\n',inputVectors)

原始输入矩阵：
 [[ 2.15537735  0.22705189 -0.70779433]
 [ 0.5565956  -0.63958618 -0.4353074 ]
 [-0.45958707  0.39059256 -0.64405861]
 [-0.87617474  0.33549768  0.01753119]
 [-1.25414553 -1.20423598  0.67677711]]


In [21]:
print('原始输出矩阵:\n',outputVectors)

原始输出矩阵:
 [[ 1.28493742 -1.70937664  1.85814902]
 [ 0.77149052 -0.40589001 -1.33859911]
 [-0.37693518 -0.39502208  0.65294331]
 [-0.93202538  0.18278074  0.90653578]
 [ 0.64423245  0.59439968 -0.90836285]]


In [22]:
inputVectors -= step * gin # 更新输入词向量矩阵

In [27]:
outputVectors -= step * gout
print('更新后的输入矩阵:\n',inputVectors)
print('更新后的输出矩阵:\n',outputVectors)

更新后的输入矩阵:
 [[ 2.15537735  0.22705189 -0.70779433]
 [ 0.5565956  -0.63958618 -0.4353074 ]
 [-0.48031441  0.37377936 -0.54953442]
 [-0.87617474  0.33549768  0.01753119]
 [-1.25414553 -1.20423598  0.67677711]]
更新后的输出矩阵:
 [[ 1.24311004 -1.67382851  1.79953275]
 [ 0.83970098 -0.46386052 -1.24300997]
 [-0.36771428 -0.40285872  0.66586535]
 [-0.979711    0.22330767  0.83970984]
 [ 0.65631409  0.58413177 -0.89143181]]


In [29]:
c, gin, gout = skipgram(centerword, context, tokens, inputVectors, outputVectors)

In [30]:
print('cost:\n',c)

cost:
 18.281287972550516


In [32]:
inputVectors -= step * gin # 更新输入词向量矩阵
outputVectors -= step * gout
c, gin, gout = skipgram(centerword, context, tokens, inputVectors, outputVectors)

In [33]:
print('cost:\n',c)

cost:
 17.11762821815007


In [34]:
inputVectors -= step * gin # 更新输入词向量矩阵
outputVectors -= step * gout
c, gin, gout = skipgram(centerword, context, tokens, inputVectors, outputVectors)

In [35]:
print('cost:\n',c)

cost:
 16.763198971042


In [41]:
for i in range(10000):
    inputVectors -= step * gin # 更新输入词向量矩阵
    outputVectors -= step * gout
    c, gin, gout = skipgram(centerword, context, tokens, inputVectors, outputVectors)
    if i % 1000 == 0:
        print('cost:\n',c)

cost:
 12.799054142608371
cost:
 12.799007337593503
cost:
 12.798968005375666
cost:
 12.79893451731689
cost:
 12.798905682079734
cost:
 12.79888060881913
cost:
 12.79885861854011
cost:
 12.798839184980158
cost:
 12.798821894173951
cost:
 12.798806416173777


In [49]:
xx = np.array([1,2,2,3,3,4])
print(np.log(xx))
yy = np.array([0,1,0,0,0,0])
yy.T*np.log(xx)

[0.         0.69314718 0.69314718 1.09861229 1.09861229 1.38629436]


array([0.        , 0.69314718, 0.        , 0.        , 0.        ,
       0.        ])

### tensorflow实现word2vec

In [130]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random, zipfile
import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

In [133]:
url = 'http://mattmahoney.net/dc/'
def maybe_download(filename, expected_bytes):
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print(filename)
    else:
        print(statinfo.st_size)
        raise Exception(
        'Faild to verify'+filename + 'can you '
        )
    return filename
filename = maybe_download('text8.zip',31344016)

ContentTooShortError: <urlopen error retrieval incomplete: got only 209317 out of 31344016 bytes>

In [151]:
import pandas as pd
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        print(f.namelist()[0])
        #data = pd.DataFrame(f.read(f.namelist()))
        #data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    #return data

#vocabulary = read_data('./export_sql_1558435.zip')
#print(vocabulary[:100])
#print('Data size', len(vocabulary))  # 总长度1700w


### 制作词表

In [136]:
vocabulary_size = 50000
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words -1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # Unk index is zero
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, countm, dictionary, reversed_dictionary = build_dataset(vocabulary, vocabulary_size)

del vocabulary


NameError: name 'vocabulary' is not defined

## 语言模型

* 语言模型就是判断一句话是否是正常的模型

假设文本序列是$w_1,w_2,...,w_T$中的每个词是依次生成的，我们有：
$$P(w_1,w_2,...w_T) = \prod_{t=1}^{T} P(w_t|w_1,w_2,...w_{t-1})$$

举例4个词的文本序列的概率是：
$$ P(w_1,w_2,w_3,w_4) = P(w_1)P(w_2|w_1)P(w_3|w_1,w_2)P(w_4|w_1,w_2,w_3)$$

而基于马尔科夫假设，一个词的出现只与前面n个词的有关。n为多少，得到的语言模型就是N-gram模型，1,2,3元语言模型如下表示：
$$P(w_1,w_2,w_3,w_4) = P(w_1)P(w_2)P(w_3)P(w_4)$$
$$P(w_1,w_2,w_3,w_4) = P(w_1)P(w_2|w_1)P(w_3|w_2)P(w_4|w_3)$$
$$P(w_1,w_2,w_3,w_4) = P(w_1)P(w_2|w_1)P(w_3|w_1,w_2)P(w_4|w_2,w_3)$$

> Q1:假设训练数据集中有10万个词，四元语法需要存储多少词频和多词相邻频率？
先要理解为什么要存储？因为N-Gram的实质是查询个数，只要看一个词之前出现的词时目标词束的个数即可。那么求解结果相当于查一张表格，这个表格是对应的出现的情况<br/>
四元表格必然是1-4元表格组成，那么这张表格有多大，就需要存储多少词频。<br/>
很明显，表格大小为1-4元数据累计相加。
$$10^4+10^4-1+10^4-2+10^4-3$$
而存储多词相邻概率则需要累计2-4元的相邻词出现的次数。
$$10^4-1+10^4-2+10^4-3$$

## 循环神经网络

## 利用循环神经网络创作歌词

In [70]:
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [71]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

In [75]:
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [77]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

In [78]:
X = nd.arange(10).reshape((2,5))
inputs = to_onehot(X, vocab_size)
len(inputs),inputs[0].shape

(5, (2, 1027))

In [123]:
X


[[18. 19. 20. 21. 22. 23.]
 [ 0.  1.  2.  3.  4.  5.]]
<NDArray 2x6 @cpu(0)>

In [80]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print('will use', ctx)

will use cpu(0)


* 初始化模型参数

In [85]:
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [86]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [87]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [88]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (2, 256))

In [89]:

# 本函数已保存在d2lzh包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [90]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            ctx, idx_to_char, char_to_idx)

'分开谢屋布图镇斯淡屋布图'

In [91]:
# 本函数已保存在d2lzh包中方便以后使用
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [92]:
# 本函数已保存在d2lzh包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
                (outputs, state) = rnn(inputs, state, params)
                # 拼接之后形状为(num_steps * batch_size, vocab_size)
                outputs = nd.concat(*outputs, dim=0)
                # Y的形状是(batch_size, num_steps)，转置后再变成长度为
                # batch * num_steps 的向量，这样跟输出的行一一对应
                y = Y.T.reshape((-1,))
                # 使用交叉熵损失计算平均分类误差
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.asscalar() * y.size
            n += y.size

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [93]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [94]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 69.441625, time 2.91 sec
 - 分开 我不要再想你 不知 我不能再 你么不人 你有我 别怪我 一子就人 我不想 你爱么 我不要再不 我不
 - 不分开 你不么 一沉两 我不要再不 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想
epoch 100, perplexity 9.716446, time 2.44 sec
 - 分开 一直的可栈人  爱穿了的手剩 我说要你二 三两银够不够 景色入秋 你已经离开我 不知不觉 你已经这
 - 不分开堡 我不要再想你 不知不觉 你已经离开我 不知不觉 你已经离开我 不知不觉 你已经离开我 不知不觉 
epoch 150, perplexity 2.848773, time 2.38 sec
 - 分开 一直两不截  静悔你在我有多难熬多烦恼  没有你烦 我有多烦恼  没有你在我有多难熬多烦恼  没有
 - 不分开吗 我叫你爸 你打我妈 这样种吗去 我将往事不离 如果我遇见你是一场悲剧 我可以让生命就这样过  爸
epoch 200, perplexity 1.543941, time 2.33 sec
 - 分开的凯都 我有在小来 看两银够不够 景色入秋 漫天黄沙凉棍 哼哼哈兮 如果我有轻功 飞檐走壁 为人耿直
 - 不分开简 我后你爸 你打我妈 这样对吗干嘛这样 还必让酒牵边子走 瞎 说手你烦我妈要 却小的耳濡目下 想要
epoch 250, perplexity 1.369471, time 2.33 sec
 - 分开的凯黄 我有在风静 看两银够不够 景色入秋 漫天黄沙凉棍 哼哼哈兮 如果我有轻功 飞檐走壁 为人耿直
 - 不分开期把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 是谁我不见的菸  古著的让我有动


In [95]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 64.675214, time 2.64 sec
 - 分开 我不要再想你我 我想能的爱写我人 我想我这想你 一直在 一沉两 我想我的爱写 我有你的爱写 我有你
 - 不分开 我不要的爱写 我有你的爱写 我有你的让我 一知的客我 你不能够 我有你的 我有我有 你有我有 你有
epoch 100, perplexity 7.609210, time 2.88 sec
 - 分开 我说想这是 我不能再想 我不能再想 我不 我不 我不要再想你 是你 是你开久了吧 不通 却又意考倒
 - 不分开离 你已经很我遇 这生情文 快谁用外 一人内空 你一定梦 在一定容 我有的梦 我想了带 你已不发 你
epoch 150, perplexity 2.149840, time 2.35 sec
 - 分开 我不要这样向 我知你口天小 静静话悄我都你爸 我真的看不侬 我怀你烦国小 静知不  说知经人开我 
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.305405, time 2.32 sec
 - 分开 我候开这是对 我知你悄国活 什透都靠 得使用双截棍 哼哼哈兮 如使用双截棍 哼哼哈兮 如使用双截棍
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250, perplexity 1.167078, time 2.33 sec
 - 分开 问候到 它打我的起头 我有到看医药你说肉 草原上两只敌对野牛在远方决斗 在一处被废弃的白蛦丘 站着
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生


### 创作歌词

In [97]:
from mxnet import nd
import random
import zipfile
with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')

In [103]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r',' ')

In [104]:
corpus_chars = corpus_chars[0:10000]

*corpus_chars前10K字符当做训练集*

In [107]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1027

In [108]:
corpus_chars[:10]

'想要有直升机 想要和'

In [110]:
corpus_indices = [ idx_to_char.index(i) for i in corpus_chars if i != ' ']

In [111]:
corpus_indices[:10]

[440, 274, 360, 452, 242, 566, 440, 274, 224, 700]

In [112]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]

In [117]:
corpus_indices[:10]

[440, 274, 360, 452, 242, 566, 239, 440, 274, 224]

In [116]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]

### 时序数据的采样

* 随机采样

In [118]:
# 本函数已保存在d2lzh包中方便以后使用
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None):
    # 减1是因为输出的索引是相应输入的索引加1
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]

    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield nd.array(X, ctx), nd.array(Y, ctx)

In [120]:
myseq = list(range(30))
for X, Y in data_iter_random(myseq, batch_size=2, num_steps=6):
    print('X:',X, '\nY:',Y,'\n')

X: 
[[ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11. 12.]
 [13. 14. 15. 16. 17. 18.]]
<NDArray 2x6 @cpu(0)> 

X: 
[[18. 19. 20. 21. 22. 23.]
 [ 0.  1.  2.  3.  4.  5.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[19. 20. 21. 22. 23. 24.]
 [ 1.  2.  3.  4.  5.  6.]]
<NDArray 2x6 @cpu(0)> 



* 相邻采样

In [121]:
# 本函数已保存在d2lzh包中方便以后使用
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = nd.array(corpus_indices, ctx=ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].reshape((
        batch_size, batch_len))
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [122]:
myseq = list(range(30))
for X, Y in data_iter_random(myseq, batch_size=2, num_steps=6):
    print('X:',X, '\nY:',Y,'\n')

X: 
[[ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11. 12.]
 [13. 14. 15. 16. 17. 18.]]
<NDArray 2x6 @cpu(0)> 

X: 
[[18. 19. 20. 21. 22. 23.]
 [ 0.  1.  2.  3.  4.  5.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[19. 20. 21. 22. 23. 24.]
 [ 1.  2.  3.  4.  5.  6.]]
<NDArray 2x6 @cpu(0)> 



### 在tensorflow中进行实践序列预测

### Encoder-decoder

In [13]:
import os
os.system('./nmt-master/nmt/scripts/download_iwslt15.sh')

1